In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
######### Part I: Examine changes in portfolio mix ###########
## This part uses origination data only ##

# read in origination files of 2009 and 2009
column_names=['fico', 'dt_first_pi', 'flag_fthb', 'dt_matr', 'cd_msa', 'mi_pct', 'cnt_units',
              'occpy_sts', 'cltv', 'dti', 'orig_upb', 'ltv', 'int_rt', 'channel', 'ppmt_pnlty',
              'prod_type', 'st', 'prop_type', 'zipcode', 'id_loan', 'loan_purpose',
              'orig_loan_term', 'cnt_borr', 'seller_name', 'servicer_name', 'flag_sc']

sample_pct=0.05

destine="/Users/zhixiaolin/Documents/Freddie Mac/data/historical_data1_Q"

def read_modfile(y, q):
    file=pd.read_csv(destine+str(q)+str(y)+'.txt', sep="|", names=column_names, header=None, low_memory=False)
    return file[(np.random.uniform(0, 1, len(file['id_loan'])) < sample_pct) & (file['fico'] >= 600) 
                & (file['fico'] <= 850) & (file['prop_type']=='SF')
                & (file['orig_loan_term']==360)].assign(quarter=str(y)+'Q'+str(q))

modfile_2009Q1=read_modfile(2009, 1)
modfile_2009Q2=read_modfile(2009, 2)
modfile_2009Q3=read_modfile(2009, 3)
modfile_2009Q4=read_modfile(2009, 4)

model_file=pd.concat([modfile_2009Q1, modfile_2009Q2, modfile_2009Q3, modfile_2009Q4])
model_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54170 entries, 8 to 350509
Data columns (total 27 columns):
fico              54170 non-null int64
dt_first_pi       54170 non-null int64
flag_fthb         54170 non-null object
dt_matr           54170 non-null int64
cd_msa            45248 non-null float64
mi_pct            54170 non-null int64
cnt_units         54170 non-null int64
occpy_sts         54170 non-null object
cltv              54170 non-null int64
dti               54170 non-null int64
orig_upb          54170 non-null int64
ltv               54170 non-null int64
int_rt            54170 non-null float64
channel           54170 non-null object
ppmt_pnlty        54032 non-null object
prod_type         54170 non-null object
st                54170 non-null object
prop_type         54170 non-null object
zipcode           54170 non-null float64
id_loan           54170 non-null object
loan_purpose      54170 non-null object
orig_loan_term    54170 non-null int64
cnt_borr         

In [3]:
# read in performance files of 2008 and 2009
# select only 5-year performance for each loan
perf_names=['id_loan','Period','Act_endg_upb','delq_sts','loan_age','mths_remng','repch_flag',
'flag_mod','CD_Zero_BAL','Dt_zero_BAL','New_Int_rt','Amt_Non_Int_Brng_Upb','Dt_Lst_Pi',
'MI_Recoveries','Net_Sale_Proceed','Non_MI_Recoveries','Expenses','legal_costs',
'maint_pres_costs','taxes_ins_costs','misc_costs','actual_loss','modcost','stepmod_ind']

# define chunksize to ensure enough memory for reading in files
c_size=1000

destine_perf="/Users/zhixiaolin/Documents/Freddie Mac/data/historical_data1_time_Q"

def read_perffile(y, q, origfile, perf_end):
    mylist=[]
    for chunk in pd.read_csv(destine_perf+str(q)+str(y)+'.txt', sep="|", names=perf_names, header=None, 
                             low_memory=False, error_bad_lines=False, chunksize=c_size):
        chunk=chunk[chunk["Period"] <= perf_end]
        mylist.append(chunk)
    perf_file=pd.concat(mylist, axis=0)
    return pd.merge(origfile[['id_loan', 'int_rt', 'quarter']], perf_file, how='inner', on='id_loan')

orig_perf_2009Q1=read_perffile(2009, 1, modfile_2009Q1, 201403)
orig_perf_2009Q2=read_perffile(2009, 2, modfile_2009Q2, 201406)
orig_perf_2009Q3=read_perffile(2009, 3, modfile_2009Q3, 201409)
orig_perf_2009Q4=read_perffile(2009, 4, modfile_2009Q4, 201412)

In [4]:
# combine quarterly data into one data frame
orig_perf_combined=pd.concat([orig_perf_2009Q1, orig_perf_2009Q2, orig_perf_2009Q3, orig_perf_2009Q4])

In [5]:
# define delinquency
# Collapse all delinquency buckets >= 7 to 7.  180+ days delinquency is the usual cap to use for maximum delinquency. 
orig_perf_combined['delq_sts_num']=pd.to_numeric(orig_perf_combined['delq_sts'], errors='coerce')
orig_perf_combined['delq_bucket']=np.where(orig_perf_combined['delq_sts_num'] > 6, 7, 
                                           orig_perf_combined['delq_sts_num'])

# find the earliest month that a loan reaches 180+ days delinquent
foreclosure_month=orig_perf_combined[orig_perf_combined['delq_sts_num']==7].groupby(['id_loan'])[['Period']].min().reset_index()
foreclosure_month=foreclosure_month.rename(columns={'Period': 'Period_foreclosure'})


# Once a loan has reached 180+ days in delinquency, all subsequent performance data should be deleted
# because we consider the loan has been terminated as a loss
orig_perf_delq=pd.merge(orig_perf_combined, foreclosure_month, on='id_loan', how='outer')
orig_perf_delq=orig_perf_delq.loc[~(orig_perf_delq['Period'] > orig_perf_delq['Period_foreclosure'])]

orig_perf_delq['delq_bucket'].value_counts()

0.0    2306398
1.0      13799
2.0       3116
3.0       1332
4.0        939
5.0        711
6.0        598
7.0        489
Name: delq_bucket, dtype: int64

In [6]:
# define prepayment
# find the earliest month when prepayment occurs
orig_perf_combined['prepaid']=np.where(orig_perf_combined['CD_Zero_BAL']==1.0, 1, 0)
prepaid_month=orig_perf_combined[orig_perf_combined['prepaid']==1].groupby('id_loan')[['Period']].min().reset_index()
prepaid_month=prepaid_month.rename(columns={'Period': 'Period_prepaid'})


# After a loan has been prepaid, any subsequent performance should be deleted
# because the loans is considered terminated
orig_perf_prepaid=pd.merge(orig_perf_combined, prepaid_month, on='id_loan', how='outer')
orig_perf_prepaid=orig_perf_prepaid.loc[~(orig_perf_prepaid['Period'] > orig_perf_prepaid['Period_prepaid'])]

orig_perf_prepaid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333852 entries, 0 to 2333851
Data columns (total 30 columns):
id_loan                 object
int_rt                  float64
quarter                 object
Period                  int64
Act_endg_upb            float64
delq_sts                object
loan_age                int64
mths_remng              int64
repch_flag              object
flag_mod                object
CD_Zero_BAL             float64
Dt_zero_BAL             float64
New_Int_rt              float64
Amt_Non_Int_Brng_Upb    float64
Dt_Lst_Pi               float64
MI_Recoveries           float64
Net_Sale_Proceed        object
Non_MI_Recoveries       float64
Expenses                float64
legal_costs             float64
maint_pres_costs        float64
taxes_ins_costs         float64
misc_costs              float64
actual_loss             float64
modcost                 float64
stepmod_ind             object
delq_sts_num            float64
delq_bucket             float64
prep

In [7]:
# merge prepaid file and delinquency file
perf_delq_prepaid=pd.merge(orig_perf_delq, prepaid_month, on='id_loan', how='outer')

# Any performance data after prepayment should be deleted, including delinquency history
perf_delq_prepaid=perf_delq_prepaid.loc[~(perf_delq_prepaid['Period'] > perf_delq_prepaid['Period_prepaid'])]

In [8]:
# convert all delinquency bucket from numeric to text
# because we are going to add a flag for prepayment later
perf_delq_prepaid['monthly_status']=np.where(perf_delq_prepaid['delq_bucket']==7, "charge-off", 
                                             perf_delq_prepaid['delq_bucket'].astype(str))
perf_delq_prepaid['monthly_status'].value_counts()

0.0           2306398
1.0             13799
2.0              3116
3.0              1332
4.0               939
5.0               711
6.0               598
charge-off        489
nan                95
Name: monthly_status, dtype: int64

In [9]:
# Add a flag for prepayment into the status field
perf_delq_prepaid.loc[perf_delq_prepaid['Period']==perf_delq_prepaid['Period_prepaid'], 'monthly_status']='prepaid'

perf_delq_prepaid['monthly_status'].value_counts()

0.0           2272159
prepaid         34422
1.0             13689
2.0              3088
3.0              1318
4.0               929
5.0               703
6.0               593
charge-off        481
nan                95
Name: monthly_status, dtype: int64

In [12]:
# add a statement number for performance period for each loan
perf_delq_prepaid.sort_values(['id_loan', 'Period'])
perf_delq_prepaid['Period_seq']=perf_delq_prepaid.groupby('id_loan').cumcount()+1

In [13]:
# Create a column of previous statement number
perf_delq_prepaid['Period_seq_before']=perf_delq_prepaid['Period_seq']-1
perf_delq_prepaid[['id_loan', 'Period_seq_before', 'Period_seq', 'Period']].head(100)

,id_loan,Period_seq_before,Period_seq,Period
0,F109Q1000009,0,1,200902
1,F109Q1000009,1,2,200903
2,F109Q1000009,2,3,200904
3,F109Q1000009,3,4,200905
4,F109Q1000009,4,5,200906
5,F109Q1000009,5,6,200907
6,F109Q1000009,6,7,200908
7,F109Q1000009,7,8,200909
8,F109Q1000009,8,9,200910
9,F109Q1000009,9,10,200911


In [14]:
# obtain the beginning status and ending status for each performance month for each loan
perf_delq_prepaid_two_months=pd.merge(perf_delq_prepaid[['id_loan', 'Period_seq_before', 'Period', 'monthly_status']].rename(columns={'monthly_status': 'end_status'}),
                                      perf_delq_prepaid[['id_loan', 'Period_seq', 'monthly_status']].rename(columns={'monthly_status':'begin_status', 'Period_seq':'Period_seq_current'}),
                                      left_on=['id_loan', 'Period_seq_before'], right_on=['id_loan', 'Period_seq_current'], how='outer')
perf_delq_prepaid_two_months.loc[perf_delq_prepaid_two_months['Period_seq_before']==0, 'begin_status']='0.0'
perf_delq_prepaid_two_months.head(100)            

,id_loan,Period_seq_before,Period,end_status,Period_seq_current,begin_status
0,F109Q1000009,0.0,200902.0,0.0,NaN,0.0
1,F109Q1000009,1.0,200903.0,0.0,1.0,0.0
2,F109Q1000009,2.0,200904.0,0.0,2.0,0.0
3,F109Q1000009,3.0,200905.0,0.0,3.0,0.0
4,F109Q1000009,4.0,200906.0,0.0,4.0,0.0
5,F109Q1000009,5.0,200907.0,0.0,5.0,0.0
6,F109Q1000009,6.0,200908.0,0.0,6.0,0.0
7,F109Q1000009,7.0,200909.0,0.0,7.0,0.0
8,F109Q1000009,8.0,200910.0,0.0,8.0,0.0
9,F109Q1000009,9.0,200911.0,0.0,9.0,0.0


In [15]:
# delete observations with null data for status
perf_delq_prepaid_two_months=perf_delq_prepaid_two_months.loc[~((perf_delq_prepaid_two_months['begin_status']=='nan') 
                                                                | (perf_delq_prepaid_two_months['end_status']=='nan'))]

# generate the transition matrix
transition_matrix=pd.crosstab(index=perf_delq_prepaid_two_months["begin_status"], 
                              columns=perf_delq_prepaid_two_months['end_status'], margins=True)   
transition_matrix

end_status,0.0,1.0,2.0,3.0,4.0,5.0,6.0,charge-off,prepaid,All
begin_status,,,,,,,,,,
0.0,2265626,7501,40,6,2,1,0,3,34262,2307441
1.0,5848,5664,1850,9,2,1,0,0,107,13481
2.0,434,462,1078,1042,4,0,0,0,18,3038
3.0,102,49,109,226,791,2,0,1,13,1293
4.0,77,8,10,28,116,641,1,2,9,892
5.0,39,3,1,6,14,50,550,2,5,670
6.0,32,1,0,1,0,7,40,473,8,562
charge-off,1,1,0,0,0,1,2,0,0,5
All,2272159,13689,3088,1318,929,703,593,481,34422,2327382


In [16]:
# convert the transition matrix for probabilities
transition_matrix.div(transition_matrix["All"], axis=0)

end_status,0.0,1.0,2.0,3.0,4.0,5.0,6.0,charge-off,prepaid,All
begin_status,,,,,,,,,,
0.0,0.981878,0.003251,0.000017,0.000003,8.667611e-07,4.333805e-07,0.000000,0.000001,0.014848,1.0
1.0,0.433796,0.420147,0.137230,0.000668,1.483569e-04,7.417847e-05,0.000000,0.000000,0.007937,1.0
2.0,0.142857,0.152074,0.354839,0.342989,1.316656e-03,0.000000e+00,0.000000,0.000000,0.005925,1.0
3.0,0.078886,0.037896,0.084300,0.174787,6.117556e-01,1.546790e-03,0.000000,0.000773,0.010054,1.0
4.0,0.086323,0.008969,0.011211,0.031390,1.300448e-01,7.186099e-01,0.001121,0.002242,0.010090,1.0
5.0,0.058209,0.004478,0.001493,0.008955,2.089552e-02,7.462687e-02,0.820896,0.002985,0.007463,1.0
6.0,0.056940,0.001779,0.000000,0.001779,0.000000e+00,1.245552e-02,0.071174,0.841637,0.014235,1.0
charge-off,0.200000,0.200000,0.000000,0.000000,0.000000e+00,2.000000e-01,0.400000,0.000000,0.000000,1.0
All,0.976272,0.005882,0.001327,0.000566,3.991609e-04,3.020561e-04,0.000255,0.000207,0.014790,1.0


In [17]:
# output the transition matrix into Excel
transtion_output=transition_matrix.div(transition_matrix["All"], axis=0)
transtion_output.to_csv('/Users/zhixiaolin/Documents/Freddie Mac/transtion_output.csv', sep='\t')